In [1]:
search_for = 133
start_from = 0
threads = 10
thread = 5
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.08967018127441406
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 25
all cases: 385308361
type: [1, 1, 1, 1, 133] 133
cases of this type: 312900721
100000 16.529206973303904
200000 28.7101135185596
300000 36.08669215924023
400000 35.2449634467843
500000 34.35954072051395
600000 36.44760186152871
700000 35.79670079274614
800000 35.87493496029395
900000 35.11392593250948
1000000 31.053214128891224
1100000 33.14664812378992
1200000 15.035338275899088
1300000 29.631234708536375
1400000 32.792514980148205
1500000 31.20495842821619
1600000 34.69647300107494
1700000 34.07918265819076
1800000 33.66222183739676
1900000 30.153425875502123
2000000 28.287886261676363
2100000 29.677547057270754
2200000 29.519662951497864
2300000 28.815734457742213
2400000 9.504362158543573
2500000 30.524624725455773
2600000 37.776980551907776
2700000 35.34199528228805
2800000 37.22677829179631
2900000 35.73863845699715
3000000 36.4330260

29500000 16.43609468331691
29600000 25.93353151758376
29700000 28.844022954243762
29800000 27.54500121397961
29900000 30.87790586114272
30000000 27.43932434173305
30100000 30.555615130422176
30200000 28.80002287405111
30300000 30.507031709448487
30400000 12.359318821334679
30500000 30.689396662931266
30600000 29.295905423103566
30700000 31.68590416911414
30800000 25.58101421854578
30900000 15.140463176539017
31000000 30.53612862997618
31100000 29.148060685788142
31200000 30.49828906799574
31300000 28.125502412141824
31400000 30.013512568742247
31500000 28.13547411847605
31600000 29.35592380913961
31700000 26.53017973833524
31800000 7.819552827358975
31900000 29.41454865161978
32000000 26.31424240035206
32100000 29.693350544065098
32200000 28.118625192660744
32300000 30.695175137010654
32400000 28.167642939879293
32500000 31.24151451482676
32600000 28.654101644593194
32700000 21.60898446665801
32800000 21.514153197410085
32900000 31.31214413083434
33000000 29.140089704208645
33100000 30

59300000 13.287267572341019
59400000 24.652151624105418
59500000 27.09225491425763
59600000 25.395548890432618
59700000 24.875070592236604
59800000 24.282282129119487
59900000 23.924706980093013
60000000 15.433007301594813
60100000 17.59543467343772
60200000 25.19893050036359
60300000 24.44237548068695
60400000 23.968353757655635
60500000 26.860903362009324
60600000 25.11409305133
60700000 23.22544350365827
60800000 16.643575473214398
60900000 28.838771646459183
61000000 26.950032431555506
61100000 28.41826634240906
61200000 26.61849862818514
61300000 28.571578145834216
61400000 26.48412281164302
61500000 28.251814647277513
61600000 25.46171671831933
61700000 10.021126041002454
61800000 27.01295344835777
61900000 24.669904782402153
62000000 25.857114088256225
62100000 23.995215739177485
62200000 25.310356776795174
62300000 23.75275074529126
62400000 6.005540225355588
62500000 26.0568438049771
62600000 23.129271353675634
62700000 25.714238208141634
62800000 23.494853558238173
62900000 2

89100000 24.30163013618268
89200000 23.69832896414241
89300000 24.255385964797927
89400000 24.09980079830622
89500000 25.297969645244883
89600000 23.39814860633112
89700000 25.403539095362508
89800000 22.916405653661602
89900000 24.192801607604014
90000000 21.677554655385062
90100000 13.09665638594005
90200000 18.217700320105344
90300000 20.711213510999507
90400000 23.521322846306628
90500000 21.13886732705347
90600000 9.307686693966975
90700000 20.33363467936433
90800000 20.952996112172002
90900000 22.009474185571936
91000000 20.877089963748467
91100000 10.539981859405392
91200000 21.283048266278985
91300000 21.98600868836942
91400000 23.218462559908946
91500000 23.30542157497651
91600000 24.574590499398898
91700000 23.721270645249692
91800000 25.39794578545109
91900000 23.33999796248597
92000000 24.97493016799819
92100000 23.75321689119881
92200000 24.536915491724745
92300000 22.35768920066759
92400000 23.649119073241103
92500000 7.68385195411775
92600000 20.94732207488473
92700000 2

118200000 20.02361062298664
118300000 19.603759366803274
118400000 19.245266446844624
118500000 17.10361465584836
118600000 9.485889853019062
118700000 19.552705838366627
118800000 13.53878808177238
118900000 11.520089684286893
119000000 18.814199819181663
119100000 12.283225570720127
119200000 15.478050563326226
119300000 19.482995495446907
119400000 19.21331696558153
119500000 19.596144456774624
119600000 20.013838663887665
119700000 21.979336353203898
119800000 26.56178684030955
119900000 23.121218028766584
120000000 21.161443569731503
120100000 21.62485310881319
120200000 21.78306623443996
120300000 20.99617312222292
120400000 20.327883210550155
120500000 19.86343869738356
120600000 20.593116377604705
120700000 19.522342288067758
120800000 19.385252751815194
120900000 12.637136614101477
121000000 13.88882023441438
121100000 18.879798246071278
121200000 5.598342660201691
121300000 19.50516245275229
121400000 17.235419344227914
121500000 10.690298323250213
121600000 19.62841255516572

...

261600000 5.71535273071879
261700000 9.836072236064815
261800000 9.47788970436943
261900000 9.575775428581654
262000000 9.414792888165158
262100000 9.090194583803958
262200000 9.073285735324543
262300000 7.235058188419569
262400000 4.797134685408824
262500000 8.851563337795403
262600000 9.1311341882098
262700000 9.317946651973301
262800000 9.340239685110728
262900000 9.420934974815136
263000000 9.811186933370996
263100000 7.905642178253798
263200000 5.703947574196331
263300000 9.82991771461699
263400000 9.887201101701088
263500000 10.146367084138321
263600000 10.121126772838416
263700000 9.835827780040088
263800000 9.901528688449812
263900000 3.8676943922301086
264000000 10.566712049252732
264100000 9.738766900553166
264200000 9.256465170324873
264300000 9.11987378117898
264400000 9.290620001699542
264500000 8.829025051808543
264600000 8.825909719373604
264700000 3.5349542934021536
264800000 8.257068677324558
264900000 8.947816681440875
265000000 8.743650955503435
265100000 9.617389819

291000000 7.593621604898961
291100000 7.3890008646019885
291200000 7.599964508234341
291300000 7.645247813787676
291400000 7.573346957043911
291500000 7.824657167767207
291600000 2.784242977456635
291700000 7.941830400067592
291800000 7.582608593832632
291900000 4.139231082951079
292000000 6.878072517897396
292100000 7.51818369263315
292200000 7.324280071680048
292300000 7.464054909238376
292400000 7.302569917450655
292500000 7.128343852623808
292600000 7.3723749521013735
292700000 7.199802482924604
292800000 7.126294830404109
292900000 4.6082925654772
293000000 4.520642432529532
293100000 7.040232466502053
293200000 7.295610522530382
293300000 7.319629266419606
293400000 7.358438045345212
293500000 7.518075922336592
293600000 7.478872058820888
293700000 7.29258106341935
293800000 7.546002055819527
293900000 7.060589341129331
294000000 3.448793693165238
294100000 7.3264673956034905
294200000 6.028419380950619
294300000 4.893048216931105
294400000 7.442928108530528
294500000 7.303163279

319800000 2.419137401087789
319900000 2.448748466676859
320000000 2.6634593378758167
320100000 2.4267295687840105
320200000 2.6918534163348533
320300000 2.322984907985011
320400000 2.2783340951267106
320500000 2.4475497480175794
320600000 2.313786201188485
320700000 2.0420700499989866
320800000 2.314292683843221
320900000 2.0733148886057147
321000000 1.6698907761334267
321100000 1.2224369441805383
321200000 1.2137903576252083
321300000 1.2413136154120212
321400000 1.481295962881452
321500000 2.3302156506521428
321600000 2.6515646972820983
321700000 2.1480932086052595
321800000 1.9530523054918394
321900000 2.3352271822914488
322000000 2.361761304810979
322100000 2.2679968724869344
322200000 2.2802304986053796
322300000 2.574363807240772
322400000 2.42110346574217
322500000 2.4397798433423064
322600000 2.4710797500172754
322700000 2.3425200513770044
322800000 2.179657607303249
322900000 2.0840317173221448
323000000 2.093301982896451
323100000 2.4577039701957575
323200000 2.28746340398424

348400000 1.3751556140669943
348500000 1.3978269082854164
348600000 1.3399948538163635
348700000 1.4056298377362926
348800000 1.1416764356789324
348900000 1.2650967951447798
349000000 1.4571653306002874
349100000 1.4140272935864873
349200000 0.9749891927102989
349300000 0.6652614590511381
349400000 0.7212458633959332
349500000 0.7342146264083406
349600000 0.9097407335310855
349700000 1.0961639750149792
349800000 1.422648990093841
349900000 1.186909712132891
350000000 1.3137365483602643
350100000 1.354994188175163
350200000 1.2717935305398465
350300000 1.281791311874814
350400000 1.4498406787331184
350500000 1.3491247615014634
350600000 1.4781928534298274
350700000 1.3259439947892726
350800000 1.3789252496073225
350900000 1.2859598225763698
351000000 1.2479243720563986
351100000 1.2466582804110706
351200000 1.2796860472261786
351300000 1.243789771811372
351400000 1.2749978508741557
351500000 1.0485553701639658
351600000 0.8420192082900445
351700000 0.7326977090669273
351800000 0.7022111

376200000 0.050419458237559264
376300000 0.037118130175834894
376400000 0.0392140184786439
best so far: 0
type: [1, 1, 7, 1, 19] 133
cases of this type: 6385729
376500000 0.27466798618765403
376600000 0.29802394492581286
376700000 0.3077388234824366
376800000 0.26297094305670593
376900000 0.2513589280690074
377000000 0.2847956331002606
377100000 0.2607896113844249
377200000 0.19406850916237833
377300000 0.21633840942488575
377400000 0.23382363381484217
377500000 0.21581798095351656
377600000 0.20253716722355816
377700000 0.2297996206771367
377800000 0.21265013422670168
377900000 0.2050571057280375
378000000 0.19758219735917712
378100000 0.21774035544224382
378200000 0.2315357651812944
378300000 0.22354841865674985
378400000 0.19758970717257063
378500000 0.2016820386579169
378600000 0.2544922038760596
378700000 0.21034276219305592
378800000 0.197508771547191
378900000 0.04727032405296829
379000000 0.0462192572974205
379100000 0.04504572058070434
379200000 0.04537976878349052
379300000 0